<h2>Imports and constants</h2>

In [435]:
import pandas as pd
%matplotlib inline
#import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

In [436]:
data_path = 'C:\Users\Jakob\Desktop\RMIT\Practical Data Science\Assignment 2\\'

<h2>Functions</h2>

In [437]:
#def plot_cities(lons, lats):
#    plt.figure(figsize=(8,8))
#    ax = plt.axes(projection=ccrs.PlateCarree())
    #ax = plt.axes(projection=ccrs.EuroPP())
#ax.stock_img()
    #ax.coastlines()
#    plt.scatter(lons, lats,
#            color="blue", marker='o',
#             transform=ccrs.Geodetic(),
#             )
#    plt.show()

In [438]:
#plot_cities(new_df['Longitude'], new_df['Latitude'])

In [439]:
freedom = pd.read_csv(data_path + 'Happiness.csv', sep =',', decimal='.')

In [440]:
freedom = freedom[['Country', 'Freedom']]

In [441]:
freedom['Country'] = freedom['Country'].str.lower()

<h2>Merging world cities and cost of living index datasets</h2>

In [443]:
# Load world cities
cities = pd.read_csv(data_path + 'worldcitiespop.csv', sep=",")

In [444]:
# Load cost of living index
coli = pd.read_csv(data_path + 'Cost_of_living_index.csv', sep=",")

In [445]:
# Parse original "City" column to "City" and "Country" columns
# TODO the USA countries are in "NY, New York" now.
coli["City"], coli["Country"] = coli["City"].str.lower().str.split(", ", 1).str

In [446]:
# Take only first 2000 most populated cities and drop any duplicates in city names.
new_cities = cities.sort_values("Population", ascending=False)[0:2000].drop_duplicates("City")

In [447]:
# Check how many cities we have now.
new_cities.shape

(1969, 7)

In [448]:
# Merge new cities and cost of living index datasets.
new_df = pd.merge(new_cities, coli, on="City", how="inner")
new_df.shape

(394, 15)

In [449]:
df = new_df[["City", "Country_y", "Population", "Latitude", "Longitude", "Cost of Living Plus Rent Index", "Local Purchasing Power Index"]]
df.rename(columns={"Country_y": "Country"}, inplace=True)

In [450]:
df.shape

(394, 7)

In [451]:
df = pd.merge(df, freedom, on='Country', how='inner')

In [452]:
#import sys
#!{sys.executable} -m pip install pycountry

In [453]:
import pycountry

In [454]:
wages = pd.read_csv(data_path + 'average_wages.csv', sep=',', decimal='.')

In [455]:
wages = wages[wages['TIME'] == 2016].groupby('LOCATION', as_index=False).mean()

In [456]:
wages = wages[['LOCATION', 'Value']]

In [457]:
wages["Country"] = wages["LOCATION"].apply(lambda country_code: pycountry.countries.get(alpha_3=country_code).name.lower())

In [464]:
wages.rename(columns = {'Value':'Wages'}, inplace=True)

In [466]:
wages = wages[['Country', 'Wages']]

In [468]:
df = pd.merge(df, wages, on='Country', how='inner')

In [470]:
df.head()

,City,Country,Population,Latitude,Longitude,Cost of Living Plus Rent Index,Local Purchasing Power Index,Freedom,Wages
0,tokyo,japan,31480498.0,35.685000,139.751389,61.89,113.09,0.505626,39113
1,london,united kingdom,7421228.0,51.514125,-0.093689,76.10,100.22,0.508190,42835
2,birmingham,united kingdom,984336.0,52.466667,-1.916667,48.38,106.94,0.508190,42835
3,glasgow,united kingdom,610271.0,55.833333,-4.250000,48.04,121.47,0.508190,42835
4,belfast,united kingdom,585994.0,54.583333,-5.933333,43.58,100.69,0.508190,42835


<h2>TODO Merge other datasets to df</h2>

In [460]:
temperature = pd.read_csv(path + 'temperatures.csv', sep=',', decimal='.')

NameError: name 'path' is not defined

In [ ]:
temperature

In [ ]:
temperature['dt'] = pd.to_datetime(temperature['dt'])

In [ ]:
temperature.dtypes

In [ ]:
temperature['year'] = temperature.dt.dt.year
new_temperatures = temperature[temperature.year == 2012][['City', 'AverageTemperature']].groupby('City').mean()
new_temperatures